# TP 1 - Analisis de datos comerciales para entender su Composición

## Introducción

`Dataset: Datos.csv`

Este proyecto de data science se enfoca en el análisis de una base de datos alterada de una empresa comercial que realiza ventas online. La base de datos, cuidadosamente falsificada, no guarda relación con la realidad ni toma en cuenta efemérides, lo que permite analizar las ventas en un contexto controlado y simulado, proporcionando información valiosa para realizar inferencias sin comprometer la integridad de datos reales.

### Objetivo de análisis:

El objetivo principal del proyecto es analizar la composición y distribución de las ventas contenidas en el dataset, generando gráficos y análisis numéricos para responder preguntas de interés y validar o refutar hipótesis comerciales. Este análisis permitirá generar un informe detallado para el área comercial de la empresa.

El análisis se enfoca en las siguientes hipótesis iniciales:

1. **Distribución geográfica**: Dado que la sede principal de la empresa se encuentra en Córdoba, se espera que al menos el 70% de las ventas provengan de esta localidad.
2. **Concentración de ventas en tiendas**: El 60% de las operaciones deberían estar concentradas en tres de las tiendas principales (Tiendas 1, 2 y 5).
3. **Comportamiento temporal de las ventas**: Se asume que la mayoría de las ventas ocurren durante los fines de semana.
4. **Patrones horarios**: Se estima que una cantidad considerable de ventas se realizan en horarios de siesta.

#### Gráficos y análisis de variables:

Para dar respuesta a estas hipótesis, se crearán gráficos que incluyan al menos tres variables clave, como **ubicación geográfica**, **horario de la transacción**, y **categoría del producto**. Estos gráficos permitirán realizar un diagnóstico visual de cómo se comportan las ventas en función de las preguntas de interés. Además, el análisis numérico complementará los gráficos, brindando información cuantitativa precisa sobre las tendencias observadas.

#### Identificación de valores perdidos:

Durante el proceso de análisis, se identificarán los valores perdidos en el dataset, lo que permitirá evaluar su impacto en los resultados. Se implementarán técnicas para gestionar estos valores, garantizando que no afecten de forma significativa las conclusiones del análisis.

### Diccionario de datos:

Para facilitar la comprensión y análisis del dataset, se incluye un diccionario de datos que detalla los campos más relevantes. Este diccionario abarca variables como el **origen del pedido** (marketplace o fulfillment), el **documento del cliente**, la **ciudad** y **estado** desde donde se realizó el pedido, el **tipo de dirección de entrega**, así como detalles sobre las **promociones aplicadas**, el **código de referencia del SKU**, el **precio de venta del SKU**, entre otros.

## Diccionario

* Origin:  indica si el origen del pedido es marketplace o fulfillment.
* Sequence:  es la secuencia numérica de la transacción del pedido.
* Creation Date:  es la fecha y hora en que el cliente realizó el pedido.
* Client Document:  es el número de identificación fiscal del cliente que realizó el pedido.
* UF:  es el estado, provincia, región o departamento desde el que el cliente realizó el pedido.
* City:  es la ciudad desde la que el cliente realizó el pedido.
* Address Type:  es el tipo de dirección de entrega (domicilio o recogida).
* Postal Code:  es el código postal de la dirección de entrega.
* Courier:  es la transportadora responsable de la entrega.
* Status:  es el status del pedido en el momento de exportación la plantilla.
* Coupon:  es el ID del cupón del pedido, si lo hay.
* Quantity_SKU:  es la cantidad de ítems del mismo SKU.
* Category Ids Sku:  son los ID de las categorías en los que se encuentra el SKU. Esta información se presenta siguiendo el árbol de departamento/categoría/subcategoría. Por ejemplo, si el valor es /1000003/1000016/, eso significa que el SKU está en la categoría 1000016, que a su vez forma parte del departamento 1000003.
* Reference Code:  es el código de referencia del SKU.
* SKU Selling Price:  es el precio al que se vendió el SKU realmente, ya con los descuentos o promociones aplicados.
* SKU Total Price:  es el valor total del SKU en el pedido, considerando todos os ítems del SKU.
* Total Value:  es el valor total del pedido. Atención
* Discounts Names:  son las promociones aplicadas al pedido.
* Canceled By:  es el email del usuario que realizó la cancelación del pedido.
* Corporate Document:  es el número de registro de persona jurídica, cuando el pedido es realizado por una empresa.
* SalesChannel:  es el ID de la política comercial del pedido.


## Código

### Inicializando Datos

In [34]:
# Importación de librerias
import pandas as pd
import numpy as np
import locale
import unidecode
import matplotlib.pyplot as plt
import seaborn as sns

locale.setlocale(locale.LC_TIME, 'Spanish_Argentina.1252')

'Spanish_Argentina.1252'

In [35]:
# Asignacion del dataset a la variable "dataset"
dataset = pd.read_csv('./Datos/Datos.csv',sep=';')

In [36]:
# Lectura basica de la cabecera
dataset.head()

,Origin,Sequence,Creation Date,Client Document,UF,City,Address Type,Postal Code,Courrier,Status,...,Quantity_SKU,Category Ids Sku,Reference Code,SKU Selling Price,SKU Total Price,Total Value,Discounts Names,Cancelled By,Corporate Document,SalesChannel
0,Marketplace,1488581,2020-06-12 15:48:08Z,1,Buenos Aires,Olavarria,pickup,7400,Correo Sucursal,Faturado,...,1,/1/50/59/,1 - 1213,16999.0,16999.0,118098.00,x,NaN,NaN,1
1,Marketplace,1470647,2020-06-06 21:04:26Z,2,Córdoba,Córdoba,residential,5014,Correo,Faturado,...,1,/1/34/78/,2 - 0,8999.0,8999.0,63897.99,x,NaN,NaN,1
2,Marketplace,1464248,2020-06-01 22:38:00Z,3,Córdoba,Córdoba,residential,5000,Correo,Faturado,...,1,/1/34/78/,3 - 0,9629.1,9629.1,32108.20,x,NaN,NaN,1
3,Marketplace,1502447,2020-06-23 02:05:13Z,4,Buenos Aires,Victoria,pickup,1644,Correo Sucursal,Faturado,...,1,/1/34/78/,4 - 0,35799.0,35799.0,40299.00,NaN,NaN,NaN,1
4,Marketplace,1479789,2020-06-09 10:04:41Z,5,Buenos Aires,Santa Teresita,residential,7107,Correo,Faturado,...,1,/1/34/78/,4 - 0,33499.0,33499.0,67098.00,x,NaN,NaN,1


In [37]:
# Se obtiene información básica de las columnas
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24554 entries, 0 to 24553
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Origin              24554 non-null  object 
 1   Sequence            24554 non-null  int64  
 2   Creation Date       24554 non-null  object 
 3   Client Document     24554 non-null  int64  
 4   UF                  24554 non-null  object 
 5   City                24554 non-null  object 
 6   Address Type        24554 non-null  object 
 7   Postal Code         24554 non-null  int64  
 8   Courrier            24554 non-null  object 
 9   Status              24554 non-null  object 
 10  Coupon              609 non-null    object 
 11  Quantity_SKU        24554 non-null  int64  
 12  Category Ids Sku    24554 non-null  object 
 13  Reference Code      24554 non-null  object 
 14  SKU Selling Price   24554 non-null  float64
 15  SKU Total Price     24554 non-null  float64
 16  Tota

In [38]:
# Conversiones necesarias

# Convertir las columnas de identificadores a texto
dataset['Client Document'] = dataset['Client Document'].astype(str)
dataset['Postal Code'] = dataset['Postal Code'].astype(str)
dataset['Corporate Document'] = dataset['Corporate Document'].astype(str)
dataset['SalesChannel'] = dataset['SalesChannel'].astype(str)

# Convertir 'Creation Date' a datetime
dataset['Creation Date'] = pd.to_datetime(dataset['Creation Date'])

In [39]:
# Cambiar el nombre de columnas
dataset.rename(columns={'UF': 'State'}, inplace=True)

In [40]:
# Creación de las columnas necesarias para el analisis objetivo

# Extraer el día de la semana (Devuelve números: 0=Lunes, 6=Domingo)
dataset['Day of Week'] = dataset['Creation Date'].dt.dayofweek

# Extraer el nombre del día
dataset['Day Name'] = dataset['Creation Date'].dt.strftime('%A') 

# Extraer la hora
dataset['Hour'] = dataset['Creation Date'].dt.hour

In [41]:
# Correción de valores (ETL)

# Provincias (States)
dataset['State'] = dataset['State'].str.title()
dataset['State'] = dataset['State'].replace({
    'Cordoba': 'Córdoba',
    'Entre Rios': 'Entre Ríos',
    'Misiones': 'Misíones',
    'Neuquen': 'Neuquén',
    'Rio Negro': 'Río Negro',
    'Santa Fe': 'Santa Fé',
    'Tucuman': 'Tucumán',
    'Ciudad Autónoma De Buenos Aires': 'CABA'
})

# Ciudades (City)
dataset['City'] = dataset['City'].str.title()
dataset['City'] = dataset['City'].str.strip()
dataset['City'] = dataset['City'].str.replace(r'\s+', ' ', regex=True)
dataset['City'] = dataset['City'].apply(unidecode.unidecode)
dataset['City'] = dataset['City'].replace({
    'Ciudad Autonoma De Buenos Aires': 'CABA',
    'C.A.B.A.': 'CABA',
    'El Jaguel': 'El Jagüel',
    'Emilio Bunge': 'Emilio V. Bunge',
    'Joaquin V Gonzalez': 'Joaquin V. Gonzalez',
    'Jose C. Paz': 'Jose Clemente Paz',
    'Pto.iguazu': 'Puerto Iguazu',
    'Pto.madryn': 'Puerto Madryn'
})

In [42]:
# Revisión de valores nulos
dataset.isnull().sum()

Origin                    0
Sequence                  0
Creation Date             0
Client Document           0
State                     0
City                      0
Address Type              0
Postal Code               0
Courrier                  0
Status                    0
Coupon                23945
Quantity_SKU              0
Category Ids Sku          0
Reference Code            0
SKU Selling Price         0
SKU Total Price           0
Total Value               0
Discounts Names       13235
Cancelled By          22883
Corporate Document        0
SalesChannel              0
Day of Week               0
Day Name                  0
Hour                      0
dtype: int64

In [49]:
# TRATAMIENTO DE VALORES NULOS
dataset['Discounts Names'].fillna('Sin descuento', inplace=True)

Las columnas 'Coupon' y 'Cancelled By' esta bien que tengan valores nulos porqu no se rellenarian en su caso

In [44]:
# Renombrar Columnas al español
columnas_originales = [
    'Origin', 
    'Sequence', 
    'Creation Date', 
    'Client Document', 
    'State', 
    'City', 
    'Address Type', 
    'Postal Code', 
    'Courier', 
    'Status', 
    'Coupon', 
    'Quantity_SKU', 
    'Category Ids Sku', 
    'Reference Code', 
    'SKU Selling Price', 
    'SKU Total Price', 
    'Total Value', 
    'Discounts Names', 
    'Cancelled By', 
    'Corporate Document', 
    'SalesChannel',
    'Day of Week',
    'Day Name',
    'Hour'
]

nuevos_nombres = [
    'Origen', 
    'Secuencia', 
    'Fecha de Creación', 
    'Documento del Cliente', 
    'Provincia', 
    'Ciudad', 
    'Tipo de Dirección', 
    'Código Postal', 
    'Transportista', 
    'Estado', 
    'Cupón', 
    'Cantidad_SKU', 
    'IDs de Categoría SKU', 
    'Código de Referencia', 
    'Precio de Venta SKU', 
    'Precio Total SKU', 
    'Valor Total', 
    'Nombres de Descuentos', 
    'Cancelado Por', 
    'Documento Corporativo', 
    'Canal de Ventas',
    'Día de la Semana',
    'Día',
    'Hora'
]

# dataset.columns = nuevos_nombres


In [45]:
# Visualizacion de columnas
dataset.columns

Index(['Origin', 'Sequence', 'Creation Date', 'Client Document', 'State',
       'City', 'Address Type', 'Postal Code', 'Courrier', 'Status', 'Coupon',
       'Quantity_SKU', 'Category Ids Sku', 'Reference Code',
       'SKU Selling Price', 'SKU Total Price', 'Total Value',
       'Discounts Names', 'Cancelled By', 'Corporate Document', 'SalesChannel',
       'Day of Week', 'Day Name', 'Hour'],
      dtype='object')

In [46]:
dataset.tail()

,Origin,Sequence,Creation Date,Client Document,State,City,Address Type,Postal Code,Courrier,Status,...,SKU Selling Price,SKU Total Price,Total Value,Discounts Names,Cancelled By,Corporate Document,SalesChannel,Day of Week,Day Name,Hour
24549,Marketplace,1531456,2020-07-25 14:31:44+00:00,16372,La Rioja,La Rioja,residential,5300,Correo,Faturado,...,53099.0,53099.0,68598.0,Sin descuento,NaN,nan,4,5,sábado,14
24550,Marketplace,1529581,2020-07-21 22:27:32+00:00,17165,Buenos Aires,Moron,residential,1708,Correo,Faturado,...,56899.0,56899.0,61399.0,Sin descuento,NaN,nan,1,1,martes,22
24551,Marketplace,1525099,2020-07-16 14:27:19+00:00,16704,Buenos Aires,Juan Maria Gutierrez,residential,1890,Correo,Faturado,...,56899.0,56899.0,61399.0,Sin descuento,NaN,nan,1,3,jueves,14
24552,Marketplace,1514968,2020-07-05 18:53:56+00:00,17166,Buenos Aires,Mar Del Plata,pickup,7600,Correo Sucursal,Faturado,...,56899.0,56899.0,61399.0,Sin descuento,NaN,nan,1,6,domingo,18
24553,Marketplace,1514735,2020-07-05 14:21:16+00:00,12277,Buenos Aires,CABA,pickup,1013,Correo Sucursal,Faturado,...,9299.0,9299.0,51998.0,Sin descuento,NaN,nan,1,6,domingo,14


In [47]:
# Se obtiene nuevamente información básica de las columnas para verificación
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24554 entries, 0 to 24553
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   Origin              24554 non-null  object             
 1   Sequence            24554 non-null  int64              
 2   Creation Date       24554 non-null  datetime64[ns, UTC]
 3   Client Document     24554 non-null  object             
 4   State               24554 non-null  object             
 5   City                24554 non-null  object             
 6   Address Type        24554 non-null  object             
 7   Postal Code         24554 non-null  object             
 8   Courrier            24554 non-null  object             
 9   Status              24554 non-null  object             
 10  Coupon              609 non-null    object             
 11  Quantity_SKU        24554 non-null  int64              
 12  Category Ids Sku    24554 non-nu

In [48]:
# Analisis describe
dataset.describe()

,Sequence,Quantity_SKU,SKU Selling Price,SKU Total Price,Total Value,Day of Week,Hour
count,2.455400e+04,24554.000000,2.455400e+04,2.455400e+04,2.455400e+04,24554.000000,24554.000000
mean,1.499497e+06,1.024232,4.455358e+04,4.492515e+04,6.718265e+04,3.022481,14.376110
std,2.056837e+04,0.245398,3.898706e+04,3.911871e+04,5.315971e+04,2.069355,5.972321
min,1.463347e+06,1.000000,1.990000e+02,1.990000e+02,1.990000e+02,0.000000,0.000000
25%,1.481656e+06,1.000000,1.881000e+04,1.899925e+04,3.539900e+04,1.000000,11.000000
50%,1.500419e+06,1.000000,3.528000e+04,3.568000e+04,5.999600e+04,3.000000,15.000000
75%,1.516972e+06,1.000000,6.000000e+04,6.089900e+04,8.198475e+04,5.000000,19.000000
max,1.535422e+06,14.000000,1.119499e+06,1.119499e+06,1.119499e+06,6.000000,23.000000


Consigna

1. Generar preguntas o hipótesis de interés sobre el dataset elegido para el proyecto final.
2. Crear gráficos que usen al menos tres variables y hacer un diagnóstico de lo que los mismos nos están diciendo.
3. Vincular los gráficos y análisis numéricos hechos hasta el momento con las preguntas de interés.
4. Identificar valores perdidos.


El análisis se enfoca en las siguientes hipótesis iniciales:

1. **Distribución geográfica**: Dado que la sede principal de la empresa se encuentra en Córdoba, se espera que al menos el 70% de las ventas provengan de esta localidad.
2. **Concentración de ventas en tiendas**: El 60% de las operaciones deberían estar concentradas en tres de las tiendas principales (Tiendas 1, 2 y 5).
3. **Comportamiento temporal de las ventas**: Se asume que la mayoría de las ventas ocurren durante los fines de semana.
4. **Patrones horarios**: Se estima que una cantidad considerable de ventas se realizan en horarios de siesta.

### Gráficos y análisis de variables:

Para dar respuesta a estas hipótesis, se crearán gráficos que incluyan al menos tres variables clave, como **ubicación geográfica**, **horario de la transacción**, y **categoría del producto**. Estos gráficos permitirán realizar un diagnóstico visual de cómo se comportan las ventas en función de las preguntas de interés. Además, el análisis numérico complementará los gráficos, brindando información cuantitativa precisa sobre las tendencias observadas.

### Identificación de valores perdidos:

Durante el proceso de análisis, se identificarán los valores perdidos en el dataset, lo que permitirá evaluar su impacto en los resultados. Se implementarán técnicas para gestionar estos valores, garantizando que no afecten de forma significativa las conclusiones del análisis.